In [1]:
%pip install unstructured

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


"DOSKEY" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


In [2]:
%pip install --upgrade --quiet  docx2txt

Note: you may need to restart the kernel to use updated packages.


"DOSKEY" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


In [3]:
%pip install yandex-chain

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


"DOSKEY" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


In [6]:
%pip install doc2docx

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


"DOSKEY" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
%pip install docx2pdf

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


"DOSKEY" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
%pip install aspose-words

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/65.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/65.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/65.9 MB 435.7 kB/s eta 0:02:32
   ---------------------------------------- 0.1/65.9 MB 550.5 kB/s eta 0:02:00
   ---------------------------------------- 0.1/65.9 MB 853.3 kB/s eta 0:01:18
   ---------------------------------------- 0.2/65.9 MB 1.2 MB/s eta 0:00:55
   ---------------------------------------- 0.4/65.9 MB 1.6 MB/s eta 0:00:41
   ---------------------------------------- 0.5/65.9 MB 1.6 MB/s eta 0:00:43
   ---------------------------------------- 0.6/65.9 MB 1.8 MB/s eta 0:00:38
   ---------------------------------------- 0.8/65.9 MB 2.0 MB/s eta 0:00:34
    --------------------------------------- 1.0/65.9 MB 2.2 MB/s eta 0:00:30
    --------------------------------------- 1.1/65.9 MB 2.3 MB/s eta 0:00:29
    --

"DOSKEY" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


# Подгрузим дата-сет:

In [4]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

- Вариант с разбитеем на сигменты по рекурсии:

In [5]:
def docx_to_fragments(docx: Docx2txtLoader, spliter : RecursiveCharacterTextSplitter):
    docs = docx.load()
    fragment = spliter.create_documents([x.page_content for x in docs])    
    return fragment

In [6]:
loader = Docx2txtLoader(
    './dataset/Инструкция_пользователя_по_работе_в_системе_Naumen_Service_Desk.docx'
)

split = RecursiveCharacterTextSplitter(
    chunk_size=1024,
    chunk_overlap=128
)

In [7]:
data = docx_to_fragments(loader, split)

In [8]:
data

[Document(page_content='ИНСТРУКЦИЯ ПОЛЬЗОВАТЕЛЯ ПО РАБОТЕ С СИСТЕМОЙ NAUMEN SERVICE DESK 4.0 \n\nМосква, 2024 г.\n\nМосква, 2024 г.\n\nМосква,\n\nМосква,\n\n\nАннотация\n\nНастоящий документ разработан в рамках реализации проекта «Внедрение процесса управления заявками, инцидентами и знаниями на базе Naumen Service Desk 4.0». \n\nНастоящий документ представляет собой инструкцию пользователя по работе в системе Naumen Service Desk (далее Система).\n\nСодержание\n\nАннотация\t3\n\nСодержание\t4\n\n\t1\tСтарт работы в Системе\t6\n\n\t1.1\tВход в Систему\t6\n\n\t1.2\tИнтерфейс Системы\t6\n\n\t1.2.1\tОтображение вкладки «Мои заявки»\t7\n\n\t1.2.2\tОтображение вкладки «Карточка»\t8\n\n\t1.2.3\tОтображение вкладки «Согласования»\t9\n\n\t1.2.4\tОтображение вкладки «База знаний»\t9\n\n\t1.2.5\tОтображение вкладки «Справка по системе»\t10\n\n\t2\tОбщее описание модуля «Личный кабинет»\t11\n\n\t3\tРегистрация заявки через ЛК\t12\n\n\t4\tСогласование заявки\t14\n\n\t4.1\tРабота с перечнем согласов

# Создадим векторную базу

In [9]:
from yandex_chain import YandexEmbeddings
from langchain.vectorstores import FAISS

In [10]:
embeddings_model = YandexEmbeddings(config="config.json")

In [11]:
vectorstore = FAISS.from_texts(
    [
        i.page_content
        for i in data
    ], 
    embedding=embeddings_model
    )
retriever = vectorstore.as_retriever()

Модель для ответов:

In [12]:
from yandex_chain import YandexLLM

In [13]:
llm = YandexLLM(config="config.json")

In [14]:
from langchain_core.prompts import ChatPromptTemplate

In [15]:
def query(text):
    vect = retriever.invoke(text)
    temple = ChatPromptTemplate([
        ("system", 'Дай ответ на вопрос исходи из {vect}'),
        ("user", '{text}')
    ])
    out = llm.invoke(
        temple.invoke({
            'vect':vect,
            'text':text
        })
    )
    return out, vect

In [16]:
out = query(input())
print(out[0])
print(out[1])

Чтобы отсортировать список, необходимо выполнить следующие действия:

1. Нажмите на кнопку «Сортировка» (см. рисунок 78).
2. Откроется дополнительная панель со ссылками «изменить» и «сбросить».
3. Нажмите «изменить». Откроется форма «Настройки полей сортировки».
4. С помощью кнопок  добавьте новые атрибуты для сортировки, с помощью кнопок  настройте сортировку по убыванию/возрастанию, с помощью кнопки  удалите ненужный атрибут для сортировки.
5. После завершения настройки полей для сортировки нажмите кнопку «Сохранить».

Также вы можете отсортировать список по определённому параметру, нажав на соответствующую кнопку сортировки в списке заявок.
[Document(page_content='Рисунок 78 Кнопка сортировка\n\nДля того чтобы изменить параметры сортировки нажмите ссылку «изменить», откроется форма «Настройки полей сортировки» (см. Рисунок 79).\n\n\n\nРисунок 79 Настройка полей сортировки\n\nС помощью кнопки  добавьте новые атрибуты для сортировки, с помощью кнопок  настройте сортировку по убыванию/